In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx

In [ ]:
data = pd.read_csv('data/synth_data_for_training.csv')

## Function for mutation testing

In [ ]:
def test_mutation(data, feature_name : str, is_Fraud: bool ,value_from : int, value_to: int):
    data_modified = data.copy() 
    is_checked = 1 if is_Fraud else 0
    test_data= data_modified.loc[data_modified['checked'] == is_checked]
    test_data = test_data.loc[test_data[feature_name] == value_from]
    data_wout = test_data.copy()
    y_wout = data_wout['checked']
    X_wout= data_wout.drop(['checked'], axis=1)
    X_wout = X_wout.astype(np.float32)
    test_data[feature_name] = value_to
    y_test = test_data['checked']
    X_test= test_data.drop(['checked'], axis=1)
    X_test = X_test.astype(np.float32)
    session = rt.InferenceSession("model/model_1.onnx")
    y_pred_onnx1 =  session.run(None, {'X': X_wout.values.astype(np.float32)})
    y_pred_onnx2 =  session.run(None, {'X': X_test.values.astype(np.float32)})
    assert y_pred_onnx1[0].all() == y_pred_onnx2[0].all(), f'Model predictions are different. The model has bias towards {feature_name} with value {value_from} '
    return True


## Test Cases

In [ ]:
test_mutation(data, 'persoon_geslacht_vrouw', True, 1, 0) #test if changing gender of women who are frauds to men to see if predictions change to not fraud

In [ ]:
test_mutation(data, 'persoon_geslacht_vrouw', False, 0, 1) #test if changing gender of men who are not frauds to women to see if predictions change to not fraud

In [ ]:
test_mutation(data, 'persoonlijke_eigenschappen_taaleis_voldaan', True, 0, 1) #test if changing non dutch speaker people who are frauds to dutch speaking to see if predictions change to not fraud

In [ ]:
test_mutation(data, 'persoonlijke_eigenschappen_taaleis_voldaan', False, 1, 0) #test if changing dutch speaker people who are not frauds to non-dutch speaking to see if predictions change to fraud

In [ ]:
test_mutation(data, 'persoonlijke_eigenschappen_taaleis_voldaan', True, 2, 1) #test if changing people who are frauds and did no take the dutch exam to dutch speaking to see if predictions change to not fraud